In [2]:
pip install selenium

     ---------------------------------------- 6.9/6.9 MB 5.3 MB/s eta 0:00:00
     -------------------------------------- 384.9/384.9 kB 4.8 MB/s eta 0:00:00
  Using cached trio_websocket-0.9.2-py3-none-any.whl (16 kB)
  Using cached async_generator-1.10-py3-none-any.whl (18 kB)
  Using cached outcome-1.2.0-py2.py3-none-any.whl (9.7 kB)
     ---------------------------------------- 58.3/58.3 kB 3.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import database

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException    
import re
import pandas as pd
import time



In [8]:
# specifying path und driver

PATH = r"C:\Users\grego\Dev\Data and IR\Abstimmungsverhalten_v2\chromedriver_win32.exe"
driver = webdriver.Chrome(PATH)

C:\Users\grego\AppData\Local\Temp\ipykernel_12792\610209268.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [12]:

# Set up the webdriver
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

# Navigate to the relevant page
driver.get("https://www.bundestag.de/abgeordnete/biografien")

# Expand the page to show all MPs
button = driver.find_element(By.CSS_SELECTOR, '.bt-link-list[href="javascript:void(0);"]')
button.click()
time.sleep(5)

# Get the links of all MPs
try:
    links = [i.get_attribute('href') for i in driver.find_elements(By.CSS_SELECTOR, 'a[href^="/abgeordnete"]')]
    print('Success!!!')
    print(f'Found {len(links)} links.')
except Exception as e:
    print('Error:', e)

# Quit the webdriver
driver.quit()


Success!!!
Found 775 links.


In [9]:
# working with the links liste generated above. we need to filter out the wrong links first before iterating over it and fetching the relevant information

regex = 'biografien\/[a-zA-Z]\/'

falsche_links = []
richtige_links = []

for lnk in links:
    if bool(re.search(regex, lnk)):
        richtige_links.append(lnk)
    else:
        falsche_links.append(lnk)

print(len(falsche_links))
print(len(richtige_links))

#iterating over all  pages and getting the relevant information into df
driver = webdriver.Chrome(PATH)

#creating the lists that eventually will form the dataframe

bundestags_id = []
wahlkreis = []
nachnamen = []
vornamen = []
facebook_links = []
instagram_links = []
twitter_links = []
website_links = []


# to verify the existence of the element

def check_webpage_exists():
    try:
        driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Homepage"]')
    except NoSuchElementException:
        return False
    return True

def check_facebook_exists():
    try:
        driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Facebook"]')
    except NoSuchElementException:
        return False
    return True

def check_twitter_exists():
    try:
        driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Twitter"]')
    except NoSuchElementException:
        return False
    return True

def check_instagram_exists():
    try:
        driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Instagram"]')
    except NoSuchElementException:
        return False
    return True

def check_wahlkreis_exists():
    try:
        driver.find_element(By.CSS_SELECTOR, '.bt-link-intern[title^="Wahlkreis"]')
    except NoSuchElementException:
        return False
    return True

#for test
i = 0




for lnk in richtige_links:
    driver.get(lnk)
    
    try:
        #bundestags_id
        id_stripped = re.findall("\d+", lnk)
        bundestags_id.append(id_stripped[0])
        
        # names --- !!!! derzeit sind es nicht immer die korrekten namen. besser sie aus dem namenselement statt dem titel zu ziehen
        # NOCH UNKLAR: Welcher Name soll gewählt werden... in welchem Format. Hier scheinen Abweichungen auf der Website, bei manchen wird z.b. (Heilbronn) mitübergeben.
        get_title = driver.title
        nachnamen.append(re.findall("\w+", get_title)[-1])
        vornamen.append(re.findall("\w+", get_title)[-2])

        # wahlkreis
        if check_wahlkreis_exists() == True:
            search = driver.find_element(By.CSS_SELECTOR, '.bt-link-intern[title^="Wahlkreis"]').get_attribute('innerHTML')
            wahlkreis.append(search)
        else: #gibt mir das Bundesland
            search = driver.find_element(By.CSS_SELECTOR, '.bt-standard-content.col-sm-6.col-xs-12').get_attribute('innerHTML')
            land = re.findall('\>(.*)\<', search)
            wahlkreis.append('Landesliste:' + str(land))

        # links
        if check_webpage_exists() == True:
            element = driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Homepage"]')
            website_links.append(element.get_attribute('href'))
        else:
            website_links.append('keine auf Bundestagsseite')

        if check_facebook_exists() == True:
            element = driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Facebook"]')
            facebook_links.append(element.get_attribute('href'))
        else:
            facebook_links.append('keine auf Bundestagsseite')

        if check_twitter_exists() == True:
            element = driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Twitter"]')
            twitter_links.append(element.get_attribute('href'))
        else:
            twitter_links.append('keine auf Bundestagsseite')
            
        if check_instagram_exists() == True:
            element = driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Instagram"]')
            instagram_links.append(element.get_attribute('href'))
        else:
            instagram_links.append('keine auf Bundestagsseite')
       

    except:
        #driver.quit()
        print('duh!!!!!')
        print(error)
    
     # to test this loop with 5 iterations
    i = i + 1
    if i == 5:
        break

df_politiker_selenium = pd.DataFrame()

df_politiker_selenium['bundestags_id'] = bundestags_id
df_politiker_selenium['vorname'] = vornamen
df_politiker_selenium['nachname'] = nachnamen
df_politiker_selenium['wahlkreis'] = wahlkreis
df_politiker_selenium['facebook_links'] = facebook_links
df_politiker_selenium['twitter_links'] = twitter_links
df_politiker_selenium['website_links'] = website_links
df_politiker_selenium['instagram_links'] = instagram_links

print(df_politiker_selenium)

18
757


C:\Users\grego\AppData\Local\Temp\ipykernel_12792\3602561570.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


  bundestags_id   vorname nachname  \
0        861028     Sanae     Abdi   
1        857082     Gökay  Akbulut   
2        860100  Valentin     Abel   
3        860114        Al  Dailami   
4        860546      Knut  Abraham   

                                           wahlkreis  \
0                              Wahlkreis 093: Köln I   
1                            Wahlkreis 275: Mannheim   
2         Wahlkreis 268: Schwäbisch Hall – Hohenlohe   
3                              Wahlkreis 173: Gießen   
4  Wahlkreis 065: Elbe-Elster – Oberspreewald-Lau...   

                               facebook_links  \
0       https://www.facebook.com/sanaeabdispd   
1  https://www.facebook.com/gokay.akbulut.146   
2   https://www.facebook.com/valentinabel.fdp   
3                   keine auf Bundestagsseite   
4           https://facebook.com/knut.abraham   

                              twitter_links                  website_links  \
0     https://twitter.com/abdisanae?lang=de     https://sanae

In [24]:
# working with the links liste generated above. we need to filter out the wrong links first before iterating over it and fetching the relevant information

regex = 'biografien\/[a-zA-Z]\/'

falsche_links = []
richtige_links = []

for lnk in links:
    if bool(re.search(regex, lnk)):
        richtige_links.append(lnk)
    else:
        falsche_links.append(lnk)

print(len(falsche_links))
print(len(richtige_links))

#iterating over all  pages and getting the relevant information into df
driver = webdriver.Chrome(PATH)

#creating the lists that eventually will form the dataframe

bundestags_id = []
wahlkreis = []
nachnamen = []
vornamen = []
facebook_links = []
instagram_links = []
twitter_links = []
website_links = []


# to verify the existence of elements

def check_link_exists(link_text):
    try:
        driver.find_element(By.CSS_SELECTOR, f'.bt-link-extern[title^="{link_text}"]')
    except NoSuchElementException:
        return False
    return True


webpage_exists = check_link_exists("Homepage")
facebook_exists = check_link_exists("Facebook")
twitter_exists = check_link_exists("Twitter")
instagram_exists = check_link_exists("Instagram")
wahlkreis_exists = check_link_exists("Wahlkreis")


#for test
i = 0




for lnk in richtige_links:
    driver.get(lnk)
    
    try:
        #bundestags_id
        id_stripped = re.findall("\d+", lnk)
        bundestags_id.append(id_stripped[0])
        
        # names --- !!!! derzeit sind es nicht immer die korrekten namen. besser sie aus dem namenselement statt dem titel zu ziehen
        # NOCH UNKLAR: Welcher Name soll gewählt werden... in welchem Format. Hier scheinen Abweichungen auf der Website, bei manchen wird z.b. (Heilbronn) mitübergeben.
        get_title = driver.title
        nachnamen.append(re.findall("\w+", get_title)[-1])
        vornamen.append(re.findall("\w+", get_title)[-2])

        # wahlkreis
        if check_wahlkreis_exists() == True:
            search = driver.find_element(By.CSS_SELECTOR, '.bt-link-intern[title^="Wahlkreis"]').get_attribute('innerHTML')
            wahlkreis.append(search)
        else: #gibt mir das Bundesland
            search = driver.find_element(By.CSS_SELECTOR, '.bt-standard-content.col-sm-6.col-xs-12').get_attribute('innerHTML')
            land = re.findall('\>(.*)\<', search)
            wahlkreis.append('Landesliste:' + str(land))

        # links
        if check_webpage_exists() == True:
            element = driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Homepage"]')
            website_links.append(element.get_attribute('href'))
        else:
            website_links.append('keine auf Bundestagsseite')

        if check_facebook_exists() == True:
            element = driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Facebook"]')
            facebook_links.append(element.get_attribute('href'))
        else:
            facebook_links.append('keine auf Bundestagsseite')

        if check_twitter_exists() == True:
            element = driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Twitter"]')
            twitter_links.append(element.get_attribute('href'))
        else:
            twitter_links.append('keine auf Bundestagsseite')
            
        if check_instagram_exists() == True:
            element = driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Instagram"]')
            instagram_links.append(element.get_attribute('href'))
        else:
            instagram_links.append('keine auf Bundestagsseite')
       

    except:
        #driver.quit()
        print('duh!!!!!')
        print(error)
    
     # to test this loop with 5 iterations
    i = i + 1
    if i == 5:
        break

df_politiker_selenium = pd.DataFrame()

df_politiker_selenium['bundestags_id'] = bundestags_id
df_politiker_selenium['vorname'] = vornamen
df_politiker_selenium['nachname'] = nachnamen
df_politiker_selenium['wahlkreis'] = wahlkreis
df_politiker_selenium['facebook_links'] = facebook_links
df_politiker_selenium['twitter_links'] = twitter_links
df_politiker_selenium['website_links'] = website_links
df_politiker_selenium['instagram_links'] = instagram_links

print(df_politiker_selenium)

18
757


C:\Users\grego\AppData\Local\Temp\ipykernel_12792\2281796603.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


  bundestags_id   vorname nachname  \
0        861028     Sanae     Abdi   
1        857082     Gökay  Akbulut   
2        860100  Valentin     Abel   
3        860114        Al  Dailami   
4        860546      Knut  Abraham   

                                           wahlkreis  \
0                              Wahlkreis 093: Köln I   
1                            Wahlkreis 275: Mannheim   
2         Wahlkreis 268: Schwäbisch Hall – Hohenlohe   
3                              Wahlkreis 173: Gießen   
4  Wahlkreis 065: Elbe-Elster – Oberspreewald-Lau...   

                               facebook_links  \
0       https://www.facebook.com/sanaeabdispd   
1  https://www.facebook.com/gokay.akbulut.146   
2   https://www.facebook.com/valentinabel.fdp   
3                   keine auf Bundestagsseite   
4           https://facebook.com/knut.abraham   

                              twitter_links                  website_links  \
0     https://twitter.com/abdisanae?lang=de     https://sanae

In [21]:
class LinkFiltering:
    def __init__(self, richtige_links):
        self.links = links

    def filter_links(self):
        regex = 'biografien\/[a-zA-Z]\/'
        falsche_links = []
        richtige_links = []
        for lnk in self.links:
            if bool(re.search(regex, lnk)):
                richtige_links.append(lnk)
            else:
                falsche_links.append(lnk)
        return richtige_links
    

In [22]:
class PoliticianData:
    
    def __init__(self, richtige_links, path):
        self.links = links
        self.path = path
        self.driver = webdriver.Chrome(self.path)
        self.bundestags_id = []
        self.wahlkreis = []
        self.nachnamen = []
        self.vornamen = []
        self.facebook_links = []
        self.instagram_links = []
        self.twitter_links = []
        self.website_links = []
        
    def __del__(self):
        self.driver.quit()
        
    def filter_links(self, regex='biografien\/[a-zA-Z]\/'):
        falsche_links = []
        richtige_links = []

        for lnk in self.links:
            if bool(re.search(regex, lnk)):
                richtige_links.append(lnk)
            else:
                falsche_links.append(lnk)

        self.links = richtige_links
        
    def check_webpage_exists(self):
        try:
            self.driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Homepage"]')
        except NoSuchElementException:
            return False
        return True

    def check_facebook_exists(self):
        try:
            self.driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Facebook"]')
        except NoSuchElementException:
            return False
        return True

    def check_twitter_exists(self):
        try:
            self.driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Twitter"]')
        except NoSuchElementException:
            return False
        return True

    def check_instagram_exists(self):
        try:
            self.driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Instagram"]')
        except NoSuchElementException:
            return False
        return True

    def check_wahlkreis_exists(self):
        try:
            self.driver.find_element(By.CSS_SELECTOR, '.bt-link-intern[title^="Wahlkreis"]')
        except NoSuchElementException:
            return False
        return True
    
    def get_politician_data(self):
        self.driver.get(self.profile_link)
        
        try:
            #bundestags_id
            id_stripped = re.findall("\d+", self.profile_link)
            self.bundestags_id = id_stripped[0]

            # names --- !!!! derzeit sind es nicht immer die korrekten namen. besser sie aus dem namenselement statt dem titel zu ziehen
            # NOCH UNKLAR: Welcher Name soll gewählt werden... in welchem Format. Hier scheinen Abweichungen auf der Website, bei manchen wird z.b. (Heilbronn) mitübergeben.
            get_title = self.driver.title
            self.nachname = re.findall("\w+", get_title)[-1]
            self.vorname = re.findall("\w+", get_title)[-2]

            # wahlkreis
            if self.check_wahlkreis_exists() == True:
                search = self.driver.find_element(By.CSS_SELECTOR, '.bt-link-intern[title^="Wahlkreis"]').get_attribute('innerHTML')
                self.wahlkreis = search
            else: #gibt mir das Bundesland
                search = self.driver.find_element(By.CSS_SELECTOR, '.bt-standard-content.col-sm-6.col-xs-12').get_attribute('innerHTML')
                land = re.findall('\>(.*)\<', search)
                self.wahlkreis = 'Landesliste:' + str(land)

            # links
            if self.check_webpage_exists() == True:
                element = self.driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Homepage"]')
                self.website_links = element.get_attribute('href')
            else:
                self.website_links = 'keine auf Bundestagsseite'

            if self.check_facebook_exists() == True:
                element = self.driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Facebook"]')
                self.facebook_links = element.get_attribute('href')
            else:
                self.facebook_links = 'keine auf Bundestagsseite'

            if self.check_twitter_exists() == True:
                element = self.driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Twitter"]')
                self.twitter_links = element.get_attribute('href')
            else:
                self.twitter_links = 'keine auf Bundestagsseite'

            if self.check_instagram_exists() == True:
                element = self.driver.find_element(By.CSS_SELECTOR, '.bt-link-extern[title^="Instagram"]')
                self.instagram_links = element.get_attribute('href')
            else:
                self.instagram_links = 'keine auf Bundestagsseite'

        except Exception as e:
            print(f'Error occurred while processing profile link: {self.profile_link}.\nError message: {str(e)}')
